# BoXYZ - Ultralytics Segment YOLO Box

⚠️⚠️⚠️ WARNING ⚠️⚠️⚠️ **MAKE SURE YOU DOWNLOADED AND PROCESSED THE SCD CARTON DATASET BY [RUNNING NOTEBOOK 2.1](./2.1_download_preprocess_datasets.ipynb)**

In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 1.7 MB/s eta 0:00:00 MB/s eta 0:00:01

[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: pip install --upgrade pip


In [1]:
import os

from ultralytics import YOLO

DS_LOCATION = "/media/abawi/e38fddf9-a92e-4c73-b905-995771f8fc3a/datasets/segmentation"

## YOLOv9c Segmentation


### Pretrain on OSCD

In [ ]:
training_output_name = "train_2.2B_1_pt_oscd_yolo9c_epoch17"
model_pt = YOLO('yolov9c-seg.pt')
pt_results = model_pt.train(data=os.path.join(DS_LOCATION, 'ultralytics_carton_oscd', 'data.yaml'),
                                  name=training_output_name,
                                  epochs=17, imgsz=[600,1000], batch=8, nbs=64, freeze=0, lr0=0.001, workers=0)

### Fine-tune on LSCD (4 labels)

Following the approach in the original SCD paper, I'll fine-tune on the LSCD after training on the OSCD (fine-tuned YOLOv9)

⚠️⚠️⚠️ WARNING ⚠️⚠️⚠️ **ONLY FINE-TUNE AFTER PRETRAINING**

**NOTE**: change `training_name` to the name of your training instance `'./runs/segment/{training_name}'`

In [ ]:
training_name = "train_2.2B_1_pt_oscd_yolo9c_epoch17"
training_output_name = "train_2.2B_2_ft_lscd_yolo9c_epoch60"
model_ft = YOLO(os.path.join('runs', 'segment', training_name, 'weights', 'best.pt'))
ft_results = model_ft.train(data=os.path.join(DS_LOCATION, 'ultralytics_carton_lscd', 'data.yaml'),
                            name=training_output_name,
                            epochs=60, imgsz=[800, 1033], batch=4, nbs=4, freeze=10, lr0=0.001, workers=0, 
                            cos_lr=True, lrf=0.00001,
                            resume=False)


## Overview and results

I used the [YOLOv9c model (Generalized Efficient Layer Aggregation Network [GELAN])](https://arxiv.org/abs/2402.13616), fine-tuning it on the OSCD dataset followed by the LSCD dataset. The YOLOv9c is a "compact" variant of the YOLOv9 family that is larger (25.5M parameters) than the medium (20.1M parameters) and small (7.2M parameters) variants. I chose most hyper-parameters based on non-exhastive trial and error, on YOLOv9c's default settings, SCD's settings as reported in the [SCD paper](https://www.mdpi.com/1424-8220/22/10/3617), best-practices (previous observations done by myself), or due to resource limitation (I'm using an RTX 3080Ti to run everything locally as specified in the requirements).

**(TODO (fabawi): image defaults to 1024x1024 in yollo11, check if similar here)**

During the OSCD pretraining, none of the layers were frozen and were trained using the AdamW optimizer with a batch size of 64 (mini-batch 8 with gradient accumulation upto 64). The learning rate is automatically determined at 0.00125 with a momentum of 0.9). The remaining hyper-parameters were set to their defaults based on the YOLOv9c settings in [ultralytics](https://www.ultralytics.com/). The image size was set to [600,1000] following SCD's training+finetuning specs for the object detection task: pretraining image size was [600,1000] in the SCD paper although the task was different.

During the LSCD fine-tuning phase, 10 layers were frozen (because the images are larger and the OSCD low-level features won't have to be altered significantly to work with LSCD) and the remaining params were trained using the AdamW optimizer as well. The batch size was set to 4 (following the SCD's default mini-batch size for training models). Here we also enabled cos annealing with a final learning rate of $1e^{-5}$ since the classes and distribution of OSCD and LSCD are similar and we want to escape local minima but not catastrophically forget the features learned from OSCD.

The model was pretrained for 17 epochs on the OSCD dataset ($m\!A\!P\!_{seg} = 81.3$) followed by finetuning (resumed) for 50 epochs on the LSCD dataset ($m\!A\!P\!_{seg} = 86.5$) segmentation mask (all). This under-performs the [SCD paper's](https://www.mdpi.com/1424-8220/22/10/3617) results on the full SCD (combined evaluation and training on both OSCD and LSCD) compared to the SOLOv2 ($m\!A\!P\!_{seg} = 88.9$) and HTC ($m\!A\!P\!_{seg} = 89.6$)

## YOLOv11s Segmentation


### Pretrain on OSCD

In [2]:
training_output_name = "train_2.2B_1_pt_oscd_yolo11s_epoch17"
model_pt = YOLO('yolo11s-seg.pt')
pt_results = model_pt.train(data=os.path.join(DS_LOCATION, 'ultralytics_carton_oscd', 'data.yaml'),
                                  name=training_output_name,
                                  epochs=17, imgsz=[600,1000], batch=4, nbs=64, freeze=0, lr0=0.001, workers=0)

New https://pypi.org/project/ultralytics/8.3.126 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.123 🚀 Python-3.10.14 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12030MiB)
engine/trainer: task=segment, mode=train, model=yolo11s-seg.pt, data=/media/abawi/e38fddf9-a92e-4c73-b905-995771f8fc3a/datasets/segmentation/ultralytics_carton_oscd/data.yaml, epochs=17, time=None, patience=100, batch=4, imgsz=[600, 1000], save=True, save_period=-1, cache=False, device=None, workers=0, project=None, name=train_2.2B_1_pt_oscd_yolo11s_epoch17, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=0, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, vi

train: Scanning /media/abawi/e38fddf9-a92e-4c73-b905-995771f8fc3a/datasets/segmentation/ultralytics_carton_oscd/train/labels.cache... 7380 images, 21 backgrounds, 1 corrupt: 100%|██████████| 7401/7401 [00:00<?, ?it/s]

train: /media/abawi/e38fddf9-a92e-4c73-b905-995771f8fc3a/datasets/segmentation/ultralytics_carton_oscd/train/images/net (109).jpg: 1 duplicate labels removed
train: /media/abawi/e38fddf9-a92e-4c73-b905-995771f8fc3a/datasets/segmentation/ultralytics_carton_oscd/train/images/net (14144).jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [1.0008831]
train: /media/abawi/e38fddf9-a92e-4c73-b905-995771f8fc3a/datasets/segmentation/ultralytics_carton_oscd/train/images/net (5359).jpg: 1 duplicate labels removed
train: /media/abawi/e38fddf9-a92e-4c73-b905-995771f8fc3a/datasets/segmentation/ultralytics_carton_oscd/train/images/net (5701).jpg: 1 duplicate labels removed
train: /media/abawi/e38fddf9-a92e-4c73-b905-995771f8fc3a/datasets/segmentation/ultralytics_carton_oscd/train/images/net (5729).jpg: 1 duplicate labels removed


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2813.0±1263.2 MB/s, size: 61.2 KB)


val: Scanning /media/abawi/e38fddf9-a92e-4c73-b905-995771f8fc3a/datasets/segmentation/ultralytics_carton_oscd/valid/labels.cache... 995 images, 5 backgrounds, 0 corrupt: 100%|██████████| 1000/1000 [00:00<?, ?it/s]

...

WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):   2%|▏         | 2/125 [00:00<00:47,  2.56it/s]

WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 125/125 [00:30<00:00,  4.09it/s]


                   all       1000      20189      0.933      0.902      0.957       0.85      0.933      0.905      0.957      0.846
Speed: 0.2ms preprocess, 3.5ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs/segment/train_2.2B_1_pt_oscd_yolo11s_epoch17


### Fine-tune on LSCD (4 labels)

Following the approach in the original SCD paper, I'll fine-tune on the LSCD after training on the OSCD (fine-tuned YOLOv9)

⚠️⚠️⚠️ WARNING ⚠️⚠️⚠️ **ONLY FINE-TUNE AFTER PRETRAINING**

**NOTE**: change `training_name` to the name of your training instance `'./runs/segment/{training_name}'`

In [ ]:
training_name = "train_2.2B_1_pt_oscd_yolo11s_epoch17"
training_output_name = "train_2.2B_2_ft_lscd_yolo11s_epoch60"
model_ft = YOLO(os.path.join('runs', 'segment', training_name, 'weights', 'best.pt'))
ft_results = model_ft.train(data=os.path.join(DS_LOCATION, 'ultralytics_carton_lscd', 'data.yaml'),
                            name=training_output_name,
                            epochs=60, imgsz=[800, 1033], batch=4, nbs=64, freeze=5, lr0=0.001, workers=0, 
                            cos_lr=True, lrf=0.00001,
                            resume=False)


New https://pypi.org/project/ultralytics/8.3.127 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.123 🚀 Python-3.10.14 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12030MiB)
engine/trainer: task=segment, mode=train, model=runs/segment/train_2.2B_1_pt_oscd_yolo11s_epoch17/weights/best.pt, data=/media/abawi/e38fddf9-a92e-4c73-b905-995771f8fc3a/datasets/segmentation/ultralytics_carton_lscd/data.yaml, epochs=60, time=None, patience=100, batch=4, imgsz=[800, 1033], save=True, save_period=-1, cache=False, device=None, workers=0, project=None, name=train_2.2B_2_ft_lscd_yolo11s_epoch60, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=5, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, 

train: Scanning /media/abawi/e38fddf9-a92e-4c73-b905-995771f8fc3a/datasets/segmentation/ultralytics_carton_lscd/train/labels.cache... 6732 images, 2 backgrounds, 1 corrupt: 100%|██████████| 6735/6735 [00:00<?, ?it/s]

train: /media/abawi/e38fddf9-a92e-4c73-b905-995771f8fc3a/datasets/segmentation/ultralytics_carton_lscd/train/images/82.jpg: ignoring corrupt image/label: broken data stream when reading image file


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3745.0±2170.5 MB/s, size: 4544.9 KB)


val: Scanning /media/abawi/e38fddf9-a92e-4c73-b905-995771f8fc3a/datasets/segmentation/ultralytics_carton_lscd/valid/labels.cache... 1000 images, 0 backgrounds, 1 corrupt: 100%|██████████| 1000/1000 [00:00<?, ?it/s]

...


      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


      14/60      4.63G     0.4208     0.5865     0.8043     0.9045         27       1056: 100%|██████████| 1684/1684 [16:38<00:00,  1.69it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 125/125 [01:43<00:00,  1.21it/s]

                   all        999      10386      0.761       0.77      0.833      0.767      0.761       0.77      0.833      0.768



      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


      15/60       5.5G     0.4123     0.5666     0.7922     0.9018         90       1056:  63%|██████▎   | 1059/1684 [10:27<06:22,  1.64it/s]